# Задание

У каждого есть свой набор данный, который можно найти в ведомости на вкладке "Логистическая регрессия":

https://clck.ru/32BVWe

Скачиваете свой датасет с kaggle (для этого нужно авторизоваться с гугл аккаунт и скачать датасет).

**Ваша задача:**
1. Исследовать данные
2. Создать тренировочную и тестовую подвыборки
3. Построить модель логистической регрессии по своему набору данных (т.е. создать модель и обучить ее на тренировочной выборке)
4. Получить коэффиценты уравнения логистической регрессии (т.е. уравнение разделяющей поверхности). Сделать вывод какие из признаков оказались наиболее значимыми, какие нет и почему
5. Написать уравнение логистической регрессии.
6. Вывести confusion matrix для получившейся модели.
7. Вручную посчитать метрики: accuracy, recall, precision. f1.
8. Посчитать метрики с помощью библиотеки sklearn и сравнить с оценками из п.7.
9. Сделать письменный вывод получившихся результатов (хорошие или плохие получлись метрики и почему).

ВСЕ графики должны быть подписанны.
Ваша задача получить метрики как можно лучше.

In [101]:
%matplotlib inline
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from collections import Counter 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# NEW 
from sklearn.model_selection import cross_val_score 
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import numpy as np 
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn import metrics

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from google.colab import files
import seaborn as sn
from sklearn.preprocessing import LabelEncoder

In [115]:
df = pd.read_csv("income_evaluation.csv")

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              32561 non-null  int64 
 1    workclass       32561 non-null  object
 2    fnlwgt          32561 non-null  int64 
 3    education       32561 non-null  object
 4    education-num   32561 non-null  int64 
 5    marital-status  32561 non-null  object
 6    occupation      32561 non-null  object
 7    relationship    32561 non-null  object
 8    race            32561 non-null  object
 9    sex             32561 non-null  object
 10   capital-gain    32561 non-null  int64 
 11   capital-loss    32561 non-null  int64 
 12   hours-per-week  32561 non-null  int64 
 13   native-country  32561 non-null  object
 14   income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [104]:
df.columns

Index(['age', ' workclass', ' fnlwgt', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' income'],
      dtype='object')

In [37]:
def translate(df, col):
  translator = {}
  for i, item in enumerate(set(df[col])):
    translator[item] = i 
  df[col] = df[col].apply(lambda item: translator[item])
  return df

In [119]:
def translate2(df, col):
  enc = LabelEncoder()
  enc.fit_transform(df[col])
  df[col] = enc.fit_transform(df[col])
  return df

In [107]:
def prepare(df):
  df = translate2(df, " sex")
  df = translate2(df, " native-country")
  df = translate2(df, " education")
  df = translate2(df, " marital-status")
  df = translate2(df, " income")
  df = translate2(df, " workclass")
  df = translate2(df, " occupation")
  df = translate2(df, " relationship")
  df = translate2(df, " race")
  return df

In [120]:
df = prepare(df)
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0
32557,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1
32558,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0
32559,22,4,201490,11,9,4,1,3,4,1,0,0,20,39,0


In [121]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
age,1.000000,0.003787,-0.076646,-0.010508,0.036527,-0.266288,-0.020947,-0.263698,0.028718,0.088832,0.077674,0.057775,0.068756,-0.001151,0.234037
workclass,0.003787,1.000000,-0.016656,0.023513,0.052085,-0.064731,0.254892,-0.090461,0.049742,0.095981,0.033835,0.012216,0.138962,-0.007690,0.051604
fnlwgt,-0.076646,-0.016656,1.000000,-0.028145,-0.043195,0.028153,0.001597,0.008931,-0.021291,0.026858,0.000432,-0.010252,-0.018768,-0.051966,-0.009463
education,-0.010508,0.023513,-0.028145,1.000000,0.359153,-0.038407,-0.021260,-0.010876,0.014131,-0.027356,0.030046,0.016746,0.055510,0.064288,0.079317
education-num,0.036527,0.052085,-0.043195,0.359153,1.000000,-0.069304,0.109697,-0.094153,0.031838,0.012280,0.122630,0.079923,0.148123,0.050840,0.335154
marital-status,-0.266288,-0.064731,0.028153,-0.038407,-0.069304,1.000000,-0.009654,0.185451,-0.068013,-0.129314,-0.043393,-0.034187,-0.190519,-0.023819,-0.199307
occupation,-0.020947,0.254892,0.001597,-0.021260,0.109697,-0.009654,1.000000,-0.075607,0.006763,0.080296,0.025505,0.017987,0.080383,-0.012543,0.075468
relationship,-0.263698,-0.090461,0.008931,-0.010876,-0.094153,0.185451,-0.075607,1.000000,-0.116055,-0.582454,-0.057919,-0.061062,-0.248974,-0.005507,-0.250918
race,0.028718,0.049742,-0.021291,0.014131,0.031838,-0.068013,0.006763,-0.116055,1.000000,0.087204,0.011145,0.018899,0.041910,0.137852,0.071846
sex,0.088832,0.095981,0.026858,-0.027356,0.012280,-0.129314,0.080296,-0.582454,0.087204,1.000000,0.048480,0.045567,0.229309,-0.008119,0.215980


In [164]:
df = df.drop(columns = [' workclass', ' fnlwgt', ' native-country', ' race', ' occupation'])

In [123]:
df[' income'].value_counts()

0    24720
1     7841
Name:  income, dtype: int64

In [165]:
a, b = df.groupby([' income'])
df = a[1].head(7841).add(b[1], fill_value=0)
df

,age,education,education-num,marital-status,relationship,sex,capital-gain,capital-loss,hours-per-week,income
0,39.0,9.0,13.0,4.0,1.0,1.0,2174.0,0.0,40.0,0.0
1,50.0,9.0,13.0,2.0,0.0,1.0,0.0,0.0,13.0,0.0
2,38.0,11.0,9.0,0.0,1.0,1.0,0.0,0.0,40.0,0.0
3,53.0,1.0,7.0,2.0,0.0,1.0,0.0,0.0,40.0,0.0
4,28.0,9.0,13.0,2.0,5.0,0.0,0.0,0.0,40.0,0.0
...,...,...,...,...,...,...,...,...,...,...
32539,71.0,10.0,16.0,2.0,0.0,1.0,0.0,0.0,10.0,1.0
32545,39.0,7.0,12.0,2.0,5.0,0.0,0.0,0.0,20.0,1.0
32554,53.0,12.0,14.0,2.0,0.0,1.0,0.0,0.0,40.0,1.0
32557,40.0,11.0,9.0,2.0,0.0,1.0,0.0,0.0,40.0,1.0


In [166]:
X = df.drop([' income'], axis=1)
y = df[' income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,age,education,education-num,marital-status,relationship,sex,capital-gain,capital-loss,hours-per-week
10059,32.0,9.0,13.0,2.0,0.0,1.0,0.0,0.0,45.0
30684,39.0,15.0,10.0,2.0,0.0,1.0,0.0,0.0,40.0
7785,32.0,11.0,9.0,2.0,0.0,1.0,0.0,0.0,40.0
6145,44.0,11.0,9.0,2.0,0.0,1.0,0.0,0.0,45.0
9544,48.0,15.0,10.0,0.0,1.0,1.0,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...
5191,17.0,2.0,8.0,4.0,3.0,0.0,0.0,0.0,16.0
23419,40.0,10.0,16.0,2.0,0.0,1.0,0.0,0.0,40.0
5390,31.0,2.0,8.0,0.0,4.0,0.0,0.0,0.0,21.0
860,18.0,1.0,7.0,4.0,3.0,0.0,0.0,0.0,20.0


In [167]:
y_train.value_counts()

1.0    6324
0.0    6221
Name:  income, dtype: int64

In [168]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [169]:
print('y =', round(lr.intercept_[0], 4), end = ' ')
for i in range(len(lr.coef_[0])):
  print('+', round(lr.coef_[0][i], 4), f'* {X_train.columns.values[i]}',  end = ' ')

y = -0.0818 + 0.0029 * age + -0.0415 *  education + 0.1676 *  education-num + -0.4494 *  marital-status + -0.4014 *  relationship + 0.0389 *  sex + 0.0003 *  capital-gain + 0.0007 *  capital-loss + -0.0024 *  hours-per-week 

Возраст является важным фактором так как пожилие люди менее способны выполнять работу, в то время пол не является важным критерием тк любую работу могут сделать все одинаково не зависимо от пола 


In [170]:
y_pred = lr.predict(X_test)

In [171]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)

TN: 992
FP: 628
FN: 240
TP: 1277


In [172]:
# Accuracy
(tp + tn) / (tp + tn + fp + fn)

0.7233025183296142

In [173]:
accuracy_score(y_test, y_pred)

0.7233025183296142

In [174]:
# Recall 
recall = tp / (tp + fn)
recall

0.8417930125247198

In [175]:
recall_score(y_test, y_pred)

0.8417930125247198

In [176]:
# Precision 
precision = tp / (tp + fp)
precision

0.6703412073490813

In [177]:
precision_score(y_test, y_pred)

0.6703412073490813

In [178]:
# F1
f1 = 2 * precision * recall / (recall + precision)
f1

0.7463471654003507

In [179]:
f1_score(y_test, y_pred)

0.7463471654003507

Метрики Accuracy и Precision достаточно высокие, в то время как метрики Recall и F1 низкие. Это говорит о том, что данные несбалансированные 